In [1]:
import sys, subprocess

# Install Streamlit and pyngrok
subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", "streamlit", "pyngrok"], check=True)

CompletedProcess(args=['/usr/bin/python3', '-m', 'pip', 'install', '--quiet', 'streamlit', 'pyngrok'], returncode=0)

In [2]:
app_code = r'''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Configure the Streamlit page
st.set_page_config(
    page_title="SARIMA COVID Admissions Forecasting",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Title and description
st.title("🏥 SARIMA COVID Admissions Forecasting Dashboard")
st.markdown("""
This application uses a SARIMA (Seasonal AutoRegressive Integrated Moving Average) model
to forecast weekly COVID-19 hospital admissions. Upload your data or use sample data to generate forecasts.
""")

class SARIMAForecaster:
    def __init__(self, order=(2, 0, 0), seasonal_order=(1, 0, 1, 52)):
        self.order = order
        self.seasonal_order = seasonal_order
        self.model = None
        self.results = None
        self.is_fitted = False

    def prepare_data(self, df, target_column, train_split=0.8):
        if target_column not in df.columns:
            raise ValueError(f"Column '{target_column}' not found in dataset")
        train_size = int(len(df) * train_split)
        train_data = df[:train_size].copy()
        validation_data = df[train_size:].copy()
        self.train_ts = train_data[target_column].dropna()
        self.validation_ts = validation_data[target_column].dropna()
        self.train_data, self.validation_data = train_data, validation_data
        return train_data, validation_data

    def fit(self):
        self.model = SARIMAX(
            self.train_ts,
            order=self.order,
            seasonal_order=self.seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        self.results = self.model.fit(disp=False, maxiter=1000)
        self.is_fitted = True

    def validate(self):
        fc = self.results.get_forecast(steps=len(self.validation_ts))
        forecast = fc.predicted_mean
        mae = mean_absolute_error(self.validation_ts, forecast)
        rmse = np.sqrt(mean_squared_error(self.validation_ts, forecast))
        # MAPE: mean absolute percentage error
        mape = np.mean(np.abs((self.validation_ts - forecast) / self.validation_ts)) * 100
        total_forecast = forecast.sum()
        return forecast, mae, rmse, mape, total_forecast

    def forecast_future(self, steps):
        fc = self.results.get_forecast(steps=steps)
        mean = fc.predicted_mean
        ci = fc.conf_int()
        return mean, ci.iloc[:,0], ci.iloc[:,1]


# Sidebar configuration
st.sidebar.header(" Model Configuration")

with st.sidebar.expander("Settings Legend", expanded=True):
    st.markdown("""
    **ARIMA Non-Seasonal Parameters**
    - **p (AR order)**
      • Number of past values the model uses.
      • Increasing **p** makes the forecast pay more attention to longer history—can capture more complex autocorrelations but risks overfitting.
    - **d (Diff order)**
      • How many times to difference the series to make it stationary.
      • Higher **d** removes more trend; can stabilize the series but too much differencing may overdampen meaningful signals.
    - **q (MA order)**
      • Size of the moving-average window on model errors.
      • Increasing **q** smooths out noise by averaging more past errors, but too large can lag sudden changes.

    **Seasonal ARIMA Parameters**
    - **P (Seasonal AR)**
      • Number of seasonal lag terms (e.g. how many years/weeks back).
      • Higher **P** lets model revisit seasonal spikes further in the past but may overfit seasonal noise.
    - **D (Seasonal Diff)**
      • Seasonal differencing order to remove seasonal trends.
      • Increasing **D** flattens seasonality more aggressively—helpful if patterns drift over time.
    - **Q (Seasonal MA)**
      • Moving-average on seasonal errors.
      • Larger **Q** smooths seasonal fluctuations, at the risk of missing sharp spikes.
    - **S (Seasonal Period)**
      • Length of the season (e.g. 52 for weekly data over one year).
      • Must match true periodicity—if set incorrectly, seasonal effects get misaligned.

    **Data Split & Forecast Horizon**
    - **Train %**
      • Percentage of data used for model fitting.
      • Higher % → more training data but less left for validation.
    - **Future weeks**
      • How many weeks ahead to project.
      • Longer horizons increase uncertainty—forecasts become smoother and CIs wider.
    """)

# File upload & parameters
uploaded_file = st.sidebar.file_uploader("Upload your CSV", type=["csv"])
p = st.sidebar.number_input("p (AR order)", 0, 5, 2)
d = st.sidebar.number_input("d (Diff order)", 0, 2, 0)
q = st.sidebar.number_input("q (MA order)", 0, 5, 0)
P = st.sidebar.number_input("P (Seasonal AR)", 0, 3, 1)
D = st.sidebar.number_input("D (Seasonal Diff)", 0, 2, 0)
Q = st.sidebar.number_input("Q (Seasonal MA)", 0, 3, 1)
S = st.sidebar.number_input("S (Season)", 1, 52, 52)
train_split = st.sidebar.slider("Train %", 0.6, 0.9, 0.8, 0.05)
forecast_weeks = st.sidebar.slider("Future weeks", 1, 52, 12)

# Main logic
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.subheader("📋 Data Preview")
    st.dataframe(df.head(10), use_container_width=True)

    target_column = st.selectbox("Target column:", df.columns)
    forecaster = SARIMAForecaster(order=(p,d,q), seasonal_order=(P,D,Q,S))
    train_data, validation_data = forecaster.prepare_data(df, target_column, train_split)

    # Plot train vs validation
    st.subheader("Train/Validation Split")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_data.index, y=train_data[target_column], mode="lines", name="Train"))
    fig.add_trace(go.Scatter(x=validation_data.index, y=validation_data[target_column], mode="lines", name="Validation"))
    st.plotly_chart(fig, use_container_width=True)

    # Train & validate
    if st.button("Train & Validate"):
        with st.spinner("Fitting model…"):
            forecaster.fit()
        forecast_val, mae, rmse, mape, total_fc = forecaster.validate()
        st.success("Model fitted!")

        # Show metrics: MAE, RMSE, MAPE, Total Forecasted Admissions
        col1, col2, col3, col4 = st.columns(4)
        col1.metric("MAE", f"{mae:.2f}")
        col2.metric("RMSE", f"{rmse:.2f}")
        col3.metric("MAPE", f"{mape:.2f}%")
        col4.metric("Total Forecasted Admissions", f"{total_fc:.0f}")

        # Validation plot
        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=train_data.index, y=forecaster.train_ts, mode="lines", name="Train"))
        fig2.add_trace(go.Scatter(x=validation_data.index, y=forecaster.validation_ts, mode="lines", name="Actual"))
        fig2.add_trace(go.Scatter(x=validation_data.index, y=forecast_val, mode="lines", name="Forecast"))
        st.plotly_chart(fig2, use_container_width=True)

        # Future forecast
        if st.button("Forecast Future"):
            mean_fut, lower, upper = forecaster.forecast_future(forecast_weeks)
            last = df.index[-1]
            future_idx = np.arange(last+1, last+1+forecast_weeks)
            fig3 = go.Figure()
            fig3.add_trace(go.Scatter(x=df.index, y=df[target_column], mode="lines", name="Historical"))
            fig3.add_trace(go.Scatter(x=future_idx, y=mean_fut, mode="lines", name="Forecast"))
            fig3.add_trace(go.Scatter(x=future_idx, y=upper, fill=None, mode="lines", line=dict(width=0)))
            fig3.add_trace(go.Scatter(x=future_idx, y=lower, fill="tonexty", mode="lines", name="CI"))
            st.plotly_chart(fig3, use_container_width=True)

else:
    st.info(" Upload a CSV to get started.")

# Footer
st.markdown("---")
st.markdown("**Tip:** Run this in Colab with ngrok to share!")
'''
with open("sarima_app.py", "w") as f:
    f.write(app_code)


In [3]:
from pyngrok import ngrok


NGROK_AUTH_TOKEN = "2xuM4G80zTBj8QXC1XTOhrjaImW_5VgVWGmMQxntGTxKXwArU"
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [4]:
import time

# 1. Start Streamlit server in background
subprocess.Popen([sys.executable, "-m", "streamlit", "run", "sarima_app.py", "--server.port", "8501"])

# 2. Wait a few seconds for it to boot
time.sleep(5)

# 3. Open an ngrok tunnel on port 8501
public_url = ngrok.connect(8501).public_url

# 4. Print your public URL
print(f" Your app is live at: {public_url}")

 Your app is live at: https://d644-35-185-192-50.ngrok-free.app
